In [1]:
#交互式仪表盘
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
import dash
from dash import dcc, html
import plotly.express as px


# AI提示词与解读生成
core_results = f"""
1. 影响因素：人均能源消耗权重{lr_model.coef_[1]:.1f}（正向），人均GDP权重{lr_model.coef_[0]:.1f}（负向），模型R²={r2_score(y, lr_model.predict(X)):.3f}；
2. 巴黎协定效果：全球协定前年均减排{pre_rate:.1f}%，协定后{post_rate:.1f}%，速率提升{abs(post_rate/pre_rate):.1f}倍；
3. 分地区：欧洲减排最快（协定后{continent_rates['Europe'][1]:.1f}%/年），非洲最慢（协定后{continent_rates['Africa'][1]:.1f}%/年）；
4. 聚类结果：全球国家分为低/中/高排放-能耗三组；
5. 预测：2030年全球人均排放预计降至{future_ghg[-1]:.1f}吨/人。
"""

ai_prompt = f"""
请作为气候变化数据分析专家，基于以下量化结果回答问题：
{core_results}
1. 为什么人均能源消耗是CO₂排放的核心驱动因素？结合全球能源结构现状分析；
2. 巴黎协定后减排速率提升的核心原因是什么？区域差异的底层逻辑是什么？
3. 2030年排放预测结果是否符合1.5℃温控目标？需要哪些政策调整？
4. 国家聚类结果对全球气候政策制定有什么启示？
要求：回答需结合数据，逻辑清晰，每个问题回答不超过200字，语言通俗。
"""

# 模拟AI解读结果
ai_interpretation = f"""
### AI深度解读结果
1. 人均能源消耗是核心驱动因素，因全球60%以上的能源仍来自化石燃料（煤炭/石油/天然气），这类能源燃烧直接释放CO₂；可再生能源占比高的国家（如挪威），能源消耗与排放的关联度显著降低。
2. 巴黎协定后减排提速，核心是各国落地NDC自主贡献目标（如中国双碳、欧盟碳中和）；区域差异源于欧洲能源转型早、资金足，非洲则因经济发展依赖化石能源、气候资金不足导致进展缓慢。
3. 2030年预测排放（{future_ghg[-1]:.1f}吨/人）仍高于1.5℃温控目标对应的1.8吨/人，需加快化石能源退出、提升发展中国家清洁能源补贴。
4. 聚类结果提示政策需“分组施策”：高排放组严控化石能源消费，中排放组优化能源结构，低排放组避免走“先污染后治理”老路。
"""

# 保存AI文件
prompt_path = os.path.join(RESULT_DIR, "ai_analysis_prompt.txt")
with open(prompt_path, "w", encoding="utf-8") as f:
    f.write(ai_prompt)

interpretation_path = os.path.join(RESULT_DIR, "ai_analysis_interpretation.txt")
with open(interpretation_path, "w", encoding="utf-8") as f:
    f.write(ai_interpretation)

print(f"\{prompt_path}")
print(f" {interpretation_path}")

# 交互式仪表盘（适配新版Dash）
# 
cluster_dash = cluster_df[["Country", "Per_Capita_Energy", "Per_Capita_GHG", "Cluster_Name"]].copy()
# 合并历史+预测数据
history_pred = pd.DataFrame({
    "Year": list(time_df["Year"]) + list(future_years),
    "Per_Capita_GHG": list(time_df["Per_Capita_GHG"]) + list(future_ghg),
    "Type": ["历史值"]*11 + ["预测值"]*10
})
# 政策建议字典
policy_suggestions = {
    "高排放-高能耗组": "严控化石能源消费，加速风电/光伏替代，实施碳定价机制（如欧盟碳交易）",
    "中排放-中能耗组": "优化产业结构，淘汰落后产能，提升工业能源利用效率（如中国煤电改造）",
    "低排放-低能耗组": "避免依赖高耗能产业发展，直接引入清洁能源技术（如非洲光伏普及）",
    "全球层面": "建立气候资金转移机制，支持发展中国家能源转型，统一碳核算标准"
}

# 初始化Dash应用
app = dash.Dash(__name__, title="全球人均CO₂排放分析仪表盘")
server = app.server

# 仪表盘布局
app.layout = html.Div([
    # 标题
    html.H1("2010-2030全球人均CO₂排放分析仪表盘", style={"textAlign": "center", "padding": "20px"}),
    
    # 第一行：聚类散点图 + 政策建议
    html.Div([
        # 左侧聚类图
        html.Div([
            dcc.Graph(
                id="cluster-scatter",
                figure=px.scatter(
                    cluster_dash,
                    x="Per_Capita_Energy",
                    y="Per_Capita_GHG",
                    color="Cluster_Name",
                    hover_data=["Country"],
                    title="国家聚类：人均能源消耗 vs 人均CO₂排放",
                    color_discrete_map={
                        "低排放-低能耗组": "#2A9D8F",
                        "中排放-中能耗组": "#E9C46A",
                        "高排放-高能耗组": "#E76F51"
                    },
                    labels={
                        "Per_Capita_Energy": "人均能源消耗（吨标准煤/人）",
                        "Per_Capita_GHG": "人均CO₂排放（吨/人）"
                    }
                ).update_layout(plot_bgcolor="white")
            )
        ], style={"width": "60%", "display": "inline-block", "padding": "10px"}),
        
        # 右侧政策建议
        html.Div([
            html.H3("AI生成政策建议", style={"paddingLeft": "20px"}),
            html.Div([
                html.P(f"{group}：{suggestion}", style={"padding": "10px", "borderBottom": "1px solid #eee"})
                for group, suggestion in policy_suggestions.items()
            ])
        ], style={"width": "35%", "display": "inline-block", "verticalAlign": "top", "padding": "10px"})
    ]),
    
    # 第二行：历史+预测趋势图
    html.Div([
        dcc.Graph(
            id="prediction-trend",
            figure=px.line(
                history_pred,
                x="Year",
                y="Per_Capita_GHG",
                color="Type",
                title="2010-2030年全球人均CO₂排放趋势（历史+预测）",
                color_discrete_map={"历史值": "#2A9D8F", "预测值": "#E76F51"},
                labels={"Per_Capita_GHG": "人均CO₂排放（吨/人）"},
                markers=True
            ).update_layout(
                plot_bgcolor="white",
                shapes=[
                    # 标注巴黎协定年份
                    dict(
                        type="line",
                        x0=2015, x1=2015, y0=0, y1=1, yref="paper",
                        line=dict(color="#666", dash="dash")
                    )
                ],
                annotations=[
                    dict(
                        x=2015, y=0.95, yref="paper",
                        text="巴黎协定签署（2015）",
                        showarrow=True, arrowhead=1
                    )
                ]
            )
        )
    ], style={"padding": "10px"})
])

# 启动仪表盘（适配新版Dash的app.run()）
if __name__ == "__main__":
    print("仪表盘启动中... 请访问 http://127.0.0.1:8050/ 查看")
    app.run(debug=True)  # 替换原app.run_server(debug=True)，适配新版Dash

<string>:53: SyntaxWarning: invalid escape sequence '\{'
<>:53: SyntaxWarning: invalid escape sequence '\{'
<string>:53: SyntaxWarning: invalid escape sequence '\{'
<>:53: SyntaxWarning: invalid escape sequence '\{'
C:\Users\过之远\AppData\Local\Temp\ipykernel_9376\699171550.py:53: SyntaxWarning: invalid escape sequence '\{'
  print(f"\{prompt_path}")
C:\Users\过之远\AppData\Local\Temp\ipykernel_9376\699171550.py:53: SyntaxWarning: invalid escape sequence '\{'
  print(f"\{prompt_path}")


NameError: name 'lr_model' is not defined